In [ ]:
-- To identify bearish candlestick vs bullish candlestick, see:
-- https://en.wikipedia.org/wiki/Candlestick_pattern#Formation_of_candlestick

-- Task 1 
-- Find Hammer pattern

SELECT *
FROM (
    SELECT *,
           LAG(open) OVER (ORDER BY day) AS prev_open,
           LAG(close) OVER (ORDER BY day) AS prev_close
    FROM daily_ohlc
) AS sub
WHERE prev_close < prev_open   -- Previous day is bearish
  AND close > open             -- Current day is bullish
  AND high = close;            -- High price equals close price

-- Find Inverted Hammer pattern

SELECT *
FROM (
    SELECT *,
           LAG(open) OVER (ORDER BY day) AS prev_open,
           LAG(close) OVER (ORDER BY day) AS prev_close
    FROM daily_ohlc
) AS sub
WHERE prev_close < prev_open   -- Previous day is bearish
  AND close > open             -- Current day is bullish
  AND low = open;              -- Low price equals open price


In [ ]:
-- Task 2 
-- Find Engulfing Bullish pattern
INSERT INTO engulfing_bullish_patterns (day, open, high, low, close, volume)
SELECT day, open, high, low, close, volume
FROM (
    SELECT *,
           LAG(open) OVER (ORDER BY day) AS prev_open,
           LAG(close) OVER (ORDER BY day) AS prev_close
    FROM daily_ohlc
) AS sub
WHERE prev_close < prev_open  -- Previous day is bearish
  AND close > open            -- Current day is bullish
  AND open < prev_close       -- The bullish candle opens lower than the previous close
  AND close > prev_open;      -- The bullish candle fully engulfs the bearish one


-- Find Piercing Line pattern
INSERT INTO piercing_line_patterns (day, open, high, low, close, volume)
SELECT day, open, high, low, close, volume
FROM (
    SELECT *,
           LAG(open) OVER (ORDER BY day) AS prev_open,
           LAG(close) OVER (ORDER BY day) AS prev_close,
           LAG(low) OVER (ORDER BY day) AS prev_low
    FROM daily_ohlc
) AS sub
WHERE prev_close < prev_open  -- Previous day is bearish
  AND close > open            -- Current day is bullish
  AND open < prev_low         -- Bullish candle opens lower than previous low
  AND close > (prev_open + prev_close) / 2; -- Closes more than halfway into bearish candle


-- Find Morning Star pattern
INSERT INTO morning_star_patterns (day, open, high, low, close, volume)
SELECT day, open, high, low, close, volume
FROM (
    SELECT *,
        LAG(open, 1) OVER (ORDER BY day) AS d1_open,
        LAG(close, 1) OVER (ORDER BY day) AS d1_close,
        LAG(open, 2) OVER (ORDER BY day) AS d0_open,
        LAG(close, 2) OVER (ORDER BY day) AS d0_close
    FROM daily_ohlc
) d2
WHERE 
    d0_close < d0_open  -- First day is bearish
    AND d1_open < d0_close  -- Second day's candlestick body is entirely below the first day's close
    AND d1_close < d0_close
    AND d2.close > d2.open  -- Third day is bullish
    AND d2.close > (d0_open + d0_close) / 2; -- Third day's close overlaps the first day's body



In [ ]:
-- Task 2 Calculate average number of continuously bullish days following each pattern
-- (e.g., hammer, inverted hammer, morning star) + check how reliable each pattern is
-- at predicting the future stock prices

-- To simplify the data processing process, we should save the pattern data collected
-- from previous tasks into a table
-- E.g., prepend the previous sql statements with:
-- INSERT INTO hammer_patterns (day, open, high, low, close, volume)
-- INSERT INTO inverted_hammer_patterns (day, open, high, low, close, volume)
-- INSERT INTO morning_star_patterns (day, open, high, low, close, volume)

-- For implementation of "number of continuously bullish days following each pattern",
-- see `number_of_consecutive_bullish_days.sql`


In [ ]:
-- Task 3

-- Find Engulfing bearish pattern (simply the opposite of Engulfing bullish pattern)
INSERT INTO engulfing_bearish_patterns (day, open, high, low, close, volume)
SELECT day, open, high, low, close, volume
FROM (
    SELECT *,
           LAG(open) OVER (ORDER BY day) AS prev_open,
           LAG(close) OVER (ORDER BY day) AS prev_close
    FROM daily_ohlc
) AS sub
WHERE prev_close > prev_open
  AND close < open
  AND open > prev_close
  AND close < prev_open;

-- Find Evening star pattern (simply the opposite of morning star pattern)
INSERT INTO evening_star_patterns (day, open, high, low, close, volume)
SELECT day, open, high, low, close, volume
FROM (
    SELECT *,
        LAG(open, 1) OVER (ORDER BY day) AS d1_open,
        LAG(close, 1) OVER (ORDER BY day) AS d1_close,
        LAG(open, 2) OVER (ORDER BY day) AS d0_open,
        LAG(close, 2) OVER (ORDER BY day) AS d0_close
    FROM daily_ohlc
) d2
WHERE 
    d0_close > d0_open
    AND d1_open > d0_close
    AND d1_close > d0_close
    AND d2.close < d2.open
    AND d2.close < (d0_open + d0_close) / 2;



-- Find Three white soldier patterns
-- INSERT INTO three_white_soldiers_patterns (day, open, high, low, close, volume)
SELECT day, open, high, low, close, volume
FROM (
    SELECT 
        *,
        LAG(open, 1) OVER (ORDER BY day) AS prev_open,
        LAG(close, 1) OVER (ORDER BY day) AS prev_close,
        LAG(open, 2) OVER (ORDER BY day) AS prev2_open,
        LAG(close, 2) OVER (ORDER BY day) AS prev2_close
    FROM daily_ohlc
) t
WHERE 
    prev2_close > prev2_open  -- First day is bullish
    AND open BETWEEN prev2_open AND prev2_close  -- Second day opens within first day
    AND close > prev2_close  -- Second day closes higher
    AND close > open  -- Second day is bullish
    AND open BETWEEN prev_open AND prev_close  -- Third day opens within second day
    AND close > prev_close  -- Third day closes higher
    AND close > open;  -- Third day is bullish


### Optimisation for performance:

#### Query based improvements

1) Materialized View Definitions
A continuous aggregate in TimescaleDB is a materialized view that automatically refreshes and stores aggregated results efficiently over time.
Speeds up queries that use GROUP BY (e.g., OHLC calculations).


2) Leverage timescales specific query (hyperfunctions)
last(close, day)
first(open, day)

CREATE MATERIALIZED VIEW daily_ohlc_cagg
WITH (timescaledb.continuous) AS
SELECT 
    time_bucket('1 day', day) AS day,
    first(open, day) AS open,
    MAX(high) AS high,
    MIN(low) AS low,
    last(close, day) AS close,
    SUM(volume) AS volume
FROM daily_ohlc
GROUP BY day;

3) SQL Scripts: For creating indexes. Place discrete columns first in created indexes, then continuous columns

CREATE INDEX ON daily_ohlc_cagg (day DESC);
CREATE INDEX ON engulfing_bearish_patterns (day DESC);
CREATE INDEX ON evening_star_patterns (day DESC);
CREATE INDEX ON three_white_soldiers_patterns (day DESC);

----------------------------------------------------------------
CREATE INDEX ON daily_ohlc_cagg (day DESC, close, open);
CREATE INDEX ON daily_ohlc_cagg (day DESC, open, close);

#### Database based improvements

4) timescaledb-tune - memory (not really disk) (postgresql.conf)

#### Memory based improvements

5) schema improvements to reduce memory usage

avoid numeric data types, use int with max 2 dp. Stocks priced above $1 or penny stocks are typically quoted to 2 decimal places (https://www.investopedia.com/terms/t/tick.asp) (use smallint a range of -32,768 to +32,767 2bytes)
The storage requirement for a numeric value in PostgreSQL is two bytes for each group of four decimal digits, plus three to eight bytes of overhead.
For 24.98437:

Total digits: 7

Groups of four digits: 2 (2498 and 4370)

Storage: (2 * 2) + 3 to 8 bytes overhead

Total: 7 to 12 bytes

6) set_chunk_time_interval to define chunks that make up no more than 25% of main memory (across all hyper tables) 25% is the size of shared buffers 
shared_buffer = 25% of RAM.


#### More tuning
7) Background workers
Background workers perform background processing for operations specific to TimescaleDB (both live queries and background jobs, all kinds of User-Defined Actions/Policies).

The background worker's settings need to be tuned to get the most out of TimescaleDB—issues often arise when worker settings are not properly set. Some of the issues we see often caused by a misconfiguration of background workers are:
User-Defined Actions are not working properly.
Continuous aggregates are not working properly.
Compression policies are not working properly.
The retention policies are not working properly.
Database size rapidly increases, due to failures in compression and the data retention policies.

- You should configure the timescaledb.max_background_workers setting to be equal to the sum of your total number of databases + the total number of concurrent background workers you want running at any given point in time.
- By default, the max_parallel_workers setting corresponds to the number of CPUs available.
- max_worker_processes should be AT LEAST 3 (required for checkpointer, WAL writer, and vacuum processes) plus the sum of the background workers and parallel workers:
max_worker_processes = 3 + timescaledb.max_background_workers + max_parallel_workers.


https://www.timescale.com/blog/timescale-parameters-you-should-know-about-and-tune-to-maximize-your-performance